## Regression Car Types



In [10]:
import numpy as np
import seaborn as sns
import pandas as pd

df = pd.read_csv("../data/Autos/autos.csv.bz2", encoding = "iso-8859-1")

df = df[df["offerType"] == "Angebot"]
df = df[df["vehicleType"] == "kleinwagen"]
df = df[df["notRepairedDamage"] == "nein"]
df = df[(df["fuelType"] == "benzin") | (df["fuelType"] == "diesel") | (df["fuelType"] == "hybrid")]
df.dropna(inplace = True)

df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
17,2016-03-20 10:25:19,Renault_Twingo_1.2_16V_Aut.,privat,Angebot,1750,control,kleinwagen,2004,automatik,75,twingo,150000,2,benzin,renault,nein,2016-03-20 00:00:00,0,65599,2016-04-06 13:16:07
23,2016-03-12 19:43:07,Stadtflitzer,privat,Angebot,450,test,kleinwagen,1997,manuell,50,arosa,150000,5,benzin,seat,nein,2016-03-12 00:00:00,0,9526,2016-03-21 01:46:11
29,2016-03-08 19:55:19,Fiat_Punto_1.2,privat,Angebot,690,test,kleinwagen,2003,manuell,60,punto,150000,3,benzin,fiat,nein,2016-03-08 00:00:00,0,86199,2016-03-09 11:45:28


In [6]:
df["gearbox"].unique()

array(['manuell', 'automatik'], dtype=object)

In [27]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

X = df[["kilometer", "yearOfRegistration", "brand", "gearbox", "fuelType"]]

cf = ColumnTransformer([
    ("brand", OneHotEncoder(drop = "first"), ["brand"]),
    ("gearbox", OneHotEncoder(drop = "first"), ["gearbox"]),
    ("fuelType", OneHotEncoder(drop = "first"), ["fuelType"])
], remainder = "passthrough")
 
cf.fit(X)

X_transformed = cf.transform(X)

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

y = df["price"]

scores = []
for i in range(0, 1000):

    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, train_size = 0.75)

    model = LinearRegression()
    model.fit(X_train, y_train)

    scores.append(model.score(X_test, y_test))
    
print(np.mean(scores))

0.6828659410639056


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, train_size = 0.75)

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [35]:
X_pred = pd.DataFrame([
    [150000, 2000, "bmw", "automatik", "benzin"]
], columns = ["kilometer", "yearOfRegistration", "brand", "gearbox", "fuelType"])

model.predict(cf.transform(X_pred))

array([4554.42046759])